# PREDICCIÓN DE LA PRODUCCIÓN DE ENERGÍA EÓLICA CON SCIKIT-LEARN


## Configuración previa

In [ ]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

packages = ['pandas', 'numpy', 'matplotlib', 'seaborn', 'scikit-learn']

for package in packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# definicion de constantes usadas a lo largo del proyecto
SEED = 100472050 # la semilla debe ser el NIA de uno de los integrantes
wind_ava = pd.read_csv("data/wind_ava.csv", index_col=0)
wind_comp = pd.read_csv("data/wind_comp.csv", index_col=0)

## EDA

### Seleccion de el molino 13 (sotavento)

In [ ]:
# la variable objetivo es la columna energy y solo queremos los datos que
# terminan en 13, agregar la columna energy a wind_ava
aux = wind_ava[wind_ava.columns[wind_ava.columns.str.endswith('13')]]
# añadir la columna energy a wind_ava
aux.insert(0, "energy", wind_ava["energy"])     
wind_ava = aux
del aux

### Exploración inicial
 - Estructura.
 - Tipos.
 - Identificación de valores faltantes.

In [ ]:
display(wind_ava.head())
print("TYPES:\n",wind_ava.dtypes)
display("NULLS:", wind_ava.isnull().sum())
# comprobar si hay columnas constantes
print(wind_ava.columns[wind_ava.nunique() == 1].tolist())

Podemos observar que todos los datos son continuos, no hay NANs ni NULLS y no hay columnas constantes.

### Estadísticas descriptivas

In [ ]:
display(wind_ava.describe())

#### Busqueda de valores atipicos

In [ ]:
num_plots = len(wind_ava.columns)
num_rows = -(-num_plots // 5)  

fig, axs = plt.subplots(num_rows, 5, figsize=(15, 3*num_rows))

axs = axs.flatten() # Aplanar el array de ejes para facilitar el acceso

for i, column in enumerate(wind_ava.columns):
    sns.boxplot(x=wind_ava[column], ax=axs[i])
    axs[i].set_title(f"Boxplot de la variable {column} en wind_ava")

# Ocultar ejes sobrantes
for j in range(i+1, len(axs)):
    axs[j].axis('off')

plt.tight_layout()
plt.show()


Creacion de un dataset SIN valores atipicos (vamos a no sobreescribir el otro para poder comparar)

In [ ]:
Q1 = wind_ava.quantile(0.25)
Q3 = wind_ava.quantile(0.75)
IQR = Q3 - Q1

# Límites para identificar valores atípicos
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identificar valores atípicos
outliers = (wind_ava < lower_bound) | (wind_ava > upper_bound)

# Tratar los valores atípicos eliminándolos del conjunto de datos
wind_ava_no_outliers = wind_ava[~outliers.any(axis=1)]

### Analisis univariado

In [ ]:
# Histogramas para cada variable
''' # Descomentar para ver los histogramas de las variables con outliers, son muy similares a los de las variables sin outliers.
plt.figure(figsize=(12, 8))
for i, column in enumerate(wind_ava.columns):
    plt.subplot(5, 5, i + 1)
    sns.histplot(wind_ava[column], kde=True)
    plt.title(column)
plt.tight_layout()
plt.show()
'''
plt.figure(figsize=(12, 8))
for i, column in enumerate(wind_ava_no_outliers.columns):
    plt.subplot(5, 5, i + 1)
    sns.histplot(wind_ava_no_outliers[column], kde=True)
    plt.title(column)
plt.tight_layout()
plt.show()


Lo mas relevante de estas graficas son, tanto los picos en 0 en cape y en fsr, que como tambien veremos mas adelante estan muy relacionados.
El hecho de que la distribucion de los valores de cape sea asi tambien hace pensar que podemos tener problemas en el futuro, ya que el modelo puede  predecir siempre cape=0 con una precision bastante alta.

### Analisis multivariado

#### Matrices de correalcion

Correlacion general

In [ ]:
# correlacion entre las variables 
correlation = wind_ava_no_outliers.corr()
# poner correlaciones en valor absoluto
correlation = correlation.abs()
# mostrar la matriz de correlacion
plt.figure(figsize=(15, 15))
sns.heatmap(correlation, annot=True, cmap="GnBu", fmt=".2f")
plt.title("Matriz de correlación de wind_ava")
plt.show()

Correlacion con la variable cape.13

In [ ]:
correlation_cape13 = abs(correlation['energy']).sort_values(ascending=False)[1:]
# plot de las correlaciones
plt.figure(figsize=(10, 5))
sns.barplot(x=correlation_cape13.index, y=correlation_cape13)
plt.title("Correlación de las variables con energy")
plt.xticks(rotation=90)
plt.show()

Correlacion de cada variable con las demas (util para saber que variables pueden ser combinadas o eliminadas ya que aportan la misma información)

In [ ]:
import pprint as pp

# Para cada columna, encuentra las variables más correlacionadas
correlation_dict = {}
n = 2 # encontrar las n variables más correlacionadas, establecer n>23 para ver todas las variables
for col in correlation.columns:
    correlation_dict[col] = correlation[col].sort_values(ascending=False)[1:n+1].to_dict()

pp.pprint(correlation_dict, compact=True)

Podemos ver que hay variables que estan muy altamente correlacionadas entre si, con correlaciones cercanas a 1, habra que decidir que hacer con estas variables ya que a priori no parecen muy utiles al aportar la misma información

## Sistemas de evaluación

### Evaluación Outer e Inner, seleccion de las medidas del Error

Como hemos podido observar en el EDA, los datos contienen una gran cantidad de datos atipicos, que podemos suponer que hay ruido en los datos, es por ello que vamos a descartar MSE como medida pues es muy probable que sobreestime el error.
Ademas los valores de cape.13 van desde 0 en la mayoria de casos a varios miles en otros, lo cual tambien es un problema con el MSE, por lo que podriamos plantearnos emplear MPSE, aun asi debido a los valores atipicos esta medida tampoco nos parece la ideal. 
Lo mismo ocurre aun que en menor medida con RMSE Y RMSPE.
Una medida que poriamos plantearnos usar es MAPE o MAE para solucionar este problema con los valores atipicos, pero nos introduce de nuevo conflicto con el gran rango de valores que toma cape.13.

Teniendo esto en cuenta consideramos que las mejores medidas del error para este caso son:
- R2
- **RMSE**

#### Creación de un modelo dummy

Basandonos en las medidas seleccionadas anteriormente vamos a crear un regresor dummy y evaluarlo.

In [ ]:

from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Primero, dividiremos los datos en entrenamiento y test
X = wind_ava.drop(columns='energy')
y = wind_ava['energy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

# Entrenar un modelo de regresión "dummy" que prediga la media
# Este modelo es útil como referencia para comparar con otros modelos en el futuro
dummy = DummyRegressor(strategy="mean")
dummy.fit(X_train, y_train)

def rmse(y_true, y_pred):
    return np.sqrt(metrics.mean_squared_error(y_true, y_pred))

dummy_rmse = rmse(y_test, dummy.predict(X_test))
dummy_r2 = metrics.r2_score(y_test, dummy.predict(X_test))
dummy_mae = metrics.mean_absolute_error(y_test, dummy.predict(X_test))

print(f"RMSE del modelo dummy: {dummy_rmse:.2f}")
print(f"R^2 del modelo dummy: {dummy_r2:.2f}")
print(f"MAE del modelo dummy: {dummy_mae:.2f}")

Como se puede observar, haciendo uso de dos modelos de regresión "dummy" se tiene a un error cuadrático medio elevado.
Este modelo lo usaremos para comparar con nuestro modelo real

## Metodo de Escalado con KNN

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#normalizacion por escala min-max
min_max = make_column_transformer(
    (MinMaxScaler(), X_train.columns)
)

#normalizacion por escala estandar
standard = make_column_transformer(
    (StandardScaler(), X_train.columns)
)


### Escalado min-max

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
import time

pipe_knn = Pipeline([
        ('preproceso', min_max), #entrada del pipeline
        ('regresor', KNeighborsRegressor()) #salida del pipeline
])

np.random.seed(SEED)

t1 = time.time()
pipe_knn.fit(X_train, y_train)
t2 = time.time()

y_test_pred = pipe_knn.predict(X_test)

rmse_knn = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_knn = metrics.r2_score(y_test, y_test_pred)
print(f"RMSE: {rmse_knn}")
print(f"R2: {r2_knn}")
print("Tiempo de entrenamiento: ", t2 - t1)

### Escalado estandar

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
import time

pipe_knn = Pipeline([
        ('preproceso', standard), #entrada del pipeline
        ('regresor', KNeighborsRegressor()) #salida del pipeline
])

np.random.seed(SEED)

t1 = time.time()
pipe_knn.fit(X_train, y_train)
t2 = time.time()

y_test_pred = pipe_knn.predict(X_test)

rmse_knn = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_knn = metrics.r2_score(y_test, y_test_pred)
print(f"RMSE: {rmse_knn}")
print(f"R2: {r2_knn}")
print("Tiempo de entrenamiento: ", t2 - t1)

El escalado min-max es el que mejor se ajusta a los datos, por lo que lo usaremos en el resto del notebook

## 1.2 puntos A continuación, se considerarán estos métodos: KNN, árboles de regresión, regresión lineal (la normal y al menos, la variante Lasso) y SVM:
a.	Se evaluarán dichos modelos con sus hiperparámetros por omisión. También se medirán los tiempos que tarda el entrenamiento.

b.	Después, se ajustarán los hiperparámetros más importantes de cada método y se obtendrá su evaluación. Medir tiempos del entrenamiento, ahora con HPO.

c.	Obtener algunas conclusiones, tales como: ¿cuál es el mejor método? ¿Cuál de los métodos básicos de aprendizaje automático es más rápido? ¿Los resultados son mejores que los regresores triviales/naive/dummy? ¿El ajuste de hiperparámetros mejora con respecto a los valores por omisión? ¿Hay algún equilibrio entre tiempo de ejecución y mejora de resultados? ¿Es posible extraer de alguna técnica qué atributos son más relevantes? etc.



### Evaluacion de los modelos con los parametros por defecto

#### KNN

In [ ]:
# regresor knn con parametros por defecto 
from sklearn.neighbors import KNeighborsRegressor

pipe_knn = Pipeline([
        ('preproceso', standard),
        ('regresor', KNeighborsRegressor())
])

# Entrenar el modelo
t1 = time.time()
pipe_knn.fit(X_train, y_train)
t2 = time.time()

dt_knn = t2 - t1 

# Predecir los valores de test
y_test_pred = pipe_knn.predict(X_test)

# Calcular métricas
rmse_knn = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_knn = metrics.r2_score(y_test, y_test_pred)
print(f"RMSE: {rmse_knn}")
print(f"R2: {r2_knn}")

#### Arboles de regresion

In [ ]:
# arbol de regresion con parametros por defecto
from sklearn.tree import DecisionTreeRegressor

pipe_tree = Pipeline([
        ('preproceso', standard),
        ('regresor', DecisionTreeRegressor())
])

# Entrenar el modelo
t1 = time.time()
pipe_tree.fit(X_train, y_train)
t2 = time.time()

dt_tree = t2 - t1

# Predecir los valores de test
y_test_pred = pipe_tree.predict(X_test)

# Calcular métricas
rmse_tree = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_tree = metrics.r2_score(y_test, y_test_pred)
print(f"RMSE: {rmse_tree}")
print(f"R2: {r2_tree}")

#### Regresion lineal

In [ ]:
# regresion lineal con parametros por defecto
from sklearn.linear_model import LinearRegression

pipe_lr = Pipeline([
        ('preproceso', standard),
        ('regresor', LinearRegression())
])

# Entrenar el modelo
t1 = time.time()
pipe_lr.fit(X_train, y_train)
t2 = time.time()

dt_lr = t2 - t1
# Predecir los valores de test
y_test_pred = pipe_lr.predict(X_test)

# Calcular métricas
rmse_lr = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_lr = metrics.r2_score(y_test, y_test_pred)
print(f"RMSE: {rmse_lr}")
print(f"R2: {r2_lr}")

#### Regresion Lasso

In [ ]:
# regresor lasso con parametros por defecto
from sklearn.linear_model import Lasso

pipe_lasso = Pipeline([
        ('preproceso', standard),
        ('regresor', Lasso(max_iter=10000))
])

# Entrenar el modelo
t1 = time.time()
pipe_lasso.fit(X_train, y_train)
t2 = time.time()

dt_lasso = t2 - t1
# Predecir los valores de test
y_test_pred = pipe_lasso.predict(X_test)

# Calcular métricas
rmse_lasso = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_lasso = metrics.r2_score(y_test, y_test_pred)
print(f"RMSE: {rmse_lasso}")
print(f"R2: {r2_lasso}")

#### SVM

In [ ]:
# SVM con parametros por defecto
from sklearn.svm import SVR

pipe_svm = Pipeline([
        ('preproceso', standard),
        ('regresor', SVR())
])

# Entrenar el modelo
t1 = time.time()
pipe_svm.fit(X_train, y_train)
t2 = time.time()

dt_svm = t2 - t1

# Predecir los valores de test
y_test_pred = pipe_svm.predict(X_test)

# Calcular métricas
rmse_svm = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_svm = metrics.r2_score(y_test, y_test_pred)
print(f"RMSE: {rmse_svm}")
print(f"R2: {r2_svm}")


#### Random Forest

In [ ]:
# random forest con parametros por defecto
from sklearn.ensemble import RandomForestRegressor

pipe_rf = Pipeline([
        ('preproceso', standard),
        ('regresor', RandomForestRegressor())
])

# Entrenar el modelo
t1 = time.time()
pipe_rf.fit(X_train, y_train)
t2 = time.time()

dt_rf = t2 - t1

# Predecir los valores de test
y_test_pred = pipe_rf.predict(X_test)

# Calcular métricas
rmse_rf = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_rf = metrics.r2_score(y_test, y_test_pred)
print(f"RMSE: {rmse_rf}")
print(f"R2: {r2_rf}")

### Evaluacion de los modelos con los hiperparametros ajustados
Usando RMSE para la evaluacion interna y RMSE y R2 para la externa

#### KNN

In [ ]:
# ajuste de hiperparametros para knn basandonos en rmse

from sklearn.model_selection import GridSearchCV

param_grid = {
    'regresor__n_neighbors': range(1, 21),
    'regresor__weights': ['uniform', 'distance'],
    'regresor__p': [1, 2]
}

grid_search = GridSearchCV(pipe_knn, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)

t1 = time.time()
grid_search.fit(X_train, y_train)
t2 = time.time()
dt_knn_hpo = t2 - t1

print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Calcular métricas
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
print("Evaluacion externa:")
rmse_knn_hpo = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_knn_hpo = best_model.score(X_test, y_test)
print(f"RMSE: {rmse_knn_hpo}")
print(f"R2: {r2_knn_hpo}")


#### Arboles de regresion

In [ ]:
# ajuste de hiperparametros para arbol de regresion basandonos en rmse

param_grid = {
    'regresor__max_depth': range(1, 11),
    'regresor__min_samples_split': range(2, 11),
    'regresor__min_samples_leaf': range(1, 11)
}

grid_search = GridSearchCV(pipe_tree, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
t1 = time.time()
grid_search.fit(X_train, y_train)
t2 = time.time()

dt_tree_hpo = t2 - t1

print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Calcular métricas
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
print("Evaluacion externa:")
rmse_tree_hpo = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_tree_hpo = best_model.score(X_test, y_test)
print(f"RMSE: {rmse_tree_hpo}")
print(f"R2: {r2_tree_hpo}")

#### Regresion lineal

In [ ]:
# ajuste de hiperparametros para regresion lineal basandonos en rmse

param_grid = {
    'regresor__fit_intercept': [True, False]
}

grid_search = GridSearchCV(pipe_lr, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
t1 = time.time()
grid_search.fit(X_train, y_train)
t2 = time.time()
dt_regresion_lineal_hpo = t2 - t1

print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Calcular métricas
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
print("Evaluacion externa:")
rmse_lr_hpo = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_lr_hpo = best_model.score(X_test, y_test)
print(f"RMSE: {rmse_lr_hpo}")
print(f"R2: {r2_lr_hpo}")


#### Regresion Lasso

In [ ]:
# ajuste de hiperparametros para lasso basandonos en rmse
 
param_grid = {
    'regresor__alpha': np.logspace(-4, 4, 9)
}

grid_search = GridSearchCV(pipe_lasso, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)

t1 = time.time()
grid_search.fit(X_train, y_train)
t2 = time.time()

dt_laso_hpo = t2 - t1

print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Calcular métricas
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
print("Evaluacion externa:")
rmse_lasso_hpo = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_lasso_hpo = best_model.score(X_test, y_test)
print(f"RMSE: {rmse_lasso_hpo}")
print(f"R2: {r2_lasso_hpo}")


#### SVM

In [ ]:
# ajuste de hiperparametros para svm basandonos en rmse

param_grid = {
    'regresor__C': np.logspace(-4, 4, 9),
    'regresor__gamma': np.logspace(-4, 4, 9)
}

grid_search = GridSearchCV(pipe_svm, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)

t1 = time.time()
grid_search.fit(X_train, y_train)
t2 = time.time()

dt_svm_hpo = t2 - t1

print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Calular métricas
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
print("Evaluacion externa:")
rmse_svm_hpo = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_svm_hpo = best_model.score(X_test, y_test)
print(f"RMSE: {rmse_svm_hpo}")
print(f"R2: {r2_svm_hpo}")


#### Random Forest

In [ ]:
# ajuste de hiperparametros para random forest basandonos en rmse

param_grid = {
    'regresor__n_estimators': [150, 200],
    'regresor__max_depth': [20, None],
    'regresor__min_samples_split': [5, 10],
    'regresor__min_samples_leaf': [2, 4],
    'regresor__bootstrap': [True]
}

grid_search = GridSearchCV(pipe_rf, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
# medir tiempo de entrenamiento
t1 = time.time()
grid_search.fit(X_train, y_train)
t2 = time.time()
dt_random_forest_hpo = t2 - t1
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Calcular métricas
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
print("Evaluacion externa:")
rmse_rf_hpo = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_rf_hpo = best_model.score(X_test, y_test)
print(f"RMSE: {rmse_rf_hpo}")
print(f"R2: {r2_rf_hpo}")

Habiendo obtenido los resultados de los modelos con los hiperparametros ajustados, vamos a compararlos con los resultados obtenidos con los hiperparametros por defecto para distinguir qué modelo será empleado en la predicción de la producción de energía.

In [ ]:
# Graficaremos los resultados de los modelos con los hiperparametros ajustados y los hiperparametros por defecto

# RMSE
rmse = [rmse_knn, rmse_tree, rmse_lr, rmse_lasso, rmse_svm, rmse_rf]
rmse_hpo = [rmse_knn_hpo, rmse_tree, rmse_lr_hpo, rmse_lasso_hpo, rmse_svm_hpo, rmse_rf_hpo]

# R2
r2 = [r2_knn, r2_tree, r2_lr, r2_lasso, r2_svm, r2_rf]
r2_hpo = [r2_knn_hpo, r2_tree, r2_lr_hpo, r2_lasso_hpo, r2_svm_hpo, r2_rf_hpo]

# tiempos
tiempos = [dt_knn, dt_tree, dt_lr, dt_lasso, dt_svm, dt_rf]
tiempos_hpo = [dt_knn_hpo, dt_tree_hpo, dt_regresion_lineal_hpo, dt_laso_hpo, dt_svm_hpo, dt_random_forest_hpo]


# Crear un DataFrame con los resultados
results = pd.DataFrame({
    'RMSE': rmse,
    'RMSE HPO': rmse_hpo,
    'R2': r2,
    'R2 HPO': r2_hpo,
}, index=['KNN', 'Tree', 'LR', 'Lasso', 'SVM', 'RF'])

# Graficar los resultados
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

results[['RMSE', 'RMSE HPO']].plot(kind='bar', ax=axs[0])
axs[0].set_title('RMSE de los modelos')
axs[0].set_ylabel('RMSE')
axs[0].set_xlabel('Modelo')

results[['R2', 'R2 HPO']].plot(kind='bar', ax=axs[1])
axs[1].set_title('R2 de los modelos')
axs[1].set_ylabel('R2')
axs[1].set_xlabel('Modelo')

plt.tight_layout()
plt.show()

# Graficar los tiempos de entrenamiento en horizontal, azul para los modelos con hiperparámetros por defecto y naranja para los modelos con hiperparámetros ajustados
time_results = pd.DataFrame({
    'Tiempo de entrenamiento': tiempos,
    'Tiempo de entrenamiento HPO': tiempos_hpo,
}, index=['KNN', 'Tree', 'LR', 'Lasso', 'SVM', 'RF'])

ax = time_results.plot(kind='bar', figsize=(10, 5))
plt.title('Tiempo de entrenamiento de los modelos')
ax.set_yscale('log')
plt.xlabel('Tiempo (s)')
plt.ylabel('Modelo')
plt.show()



Como se puede observar, los modelos más prometedores son el Random Forest y el SVM, destacando éste último en el significativo cambio del error con el ajuste de hiperparámetros. No obstante, el mejor modelo es el Random Forest, ya que tiene un menor error y un mayor R2.

### Selección del mejor modelo
Hemos seleccinado random forest por que a pesar de tardar mas en entrenarse es el mejor modelo en cuanto a error y R2.
Para la optimización de los hiperparámetros ibamos a usar un gridsearch, pero debido a la cantidad de datos y a la cantidad de hiperparámetros que tiene el modelo era inviable (60-100 horas).
Despues consideramos usar un randomsearch, pero los resultados obtenidos no eran satisfactorios.
Es por ello que hemos decidido utilizar optuna, que es una libreria que nos permite optimizar los hiperparámetros de una forma mas eficiente mediante una busqueda bayesiana, sin embargo sigue siendo un proceso lento, aproximadamente 7h, por lo que lo hemos ejecutado y anotado los resultados para no necesitar recalcularlo. Si se quiere replicar el proceso hay que descomentar las lineas que se señalan en el codigo y ejecutarlo.

In [ ]:
try:
    import optuna
except ImportError:
    %pip install optuna
    import optuna

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 350),
        'max_depth': trial.suggest_int('max_depth', 20, 40),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 7),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'bootstrap':  trial.suggest_categorical('bootstrap', [True])
    }
    # print(f"Trial {trial.number} - Iniciando entrenamiento con hiperparámetros: {params}")
    
    model = RandomForestRegressor(**params)
    rmse_scores = []
    for _ in range(10):  # 10-fold cross-validation
        X_fold_train, X_fold_val, y_fold_train, y_fold_val = train_test_split(X_train, y_train, test_size=0.1, random_state=SEED)
        model.fit(X_fold_train, y_fold_train)
        y_pred = model.predict(X_fold_val)
        rmse = metrics.root_mean_squared_error(y_fold_val, y_pred)
        rmse_scores.append(rmse)
    
    # print(f"Trial {trial.number} - RMSE: {np.mean(rmse_scores)}")
    return np.mean(rmse_scores)

def progress_callback(study, trial):
    print(f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}")
    print(f"Best trial so far: {study.best_trial.number}")



# WARINING: Este proceso puede tardar mucho tiempo en completarse (5-10 horas)

# study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SEED))

# print("Iniciando búsqueda de hiperparámetros...")
# descomentar la siguiente linea para ejecutar la busqueda de hiperparametros 
# study.optimize(objective, n_trials=1000, n_jobs=-1, callbacks=[progress_callback], show_progress_bar=True)
# best_params = study.best_params
# comentar la siguiente linea para ejecutar la busqueda de hiperparametros
best_params ={'n_estimators': 333, 'max_depth': 34, 'min_samples_split': 2, 'min_samples_leaf': 2, 'bootstrap': True}

print("Mejores hiperparámetros encontrados:")
print(best_params)

In [ ]:
# Entrenar el mejor modelo con los mejores hiperparámetros
import time
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

best_model = RandomForestRegressor(**best_params)
t1 = time.time()
best_model.fit(X_train, y_train)
t2 = time.time()
dt = t2 - t1
# Predecir los valores de test
y_test_pred = best_model.predict(X_test)

# Calcular métricas
rmse_rf_cv = metrics.root_mean_squared_error(y_test, y_test_pred)
r2_rf_cv = best_model.score(X_test, y_test)
print(f"RMSE: {rmse_rf_cv}")
print(f"R2: {r2_rf_cv}")
print("Tiempo de entrenamiento: ", dt)



In [ ]:
# entrenar con los mejores hiperparametros encontrados y todos los datos
best_model.fit(X, y)

In [ ]:
# exportar el modelo en un archivo pickle
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

### Medir la precision del modelo en energy alto y energy bajo

In [ ]:
# Medir la precision del modelo en energy alto y energy bajo
# dividir el conjunto de datos en dos, uno con energy alto y otro con energy bajo
# cuando la energía sea menor que el tercer cuantil, se considerará clase “baja”, y 
# cuando sea mayor, clase  “alta”.
low_energy = wind_ava[wind_ava['energy'] < wind_ava['energy'].quantile(1/3)]
high_energy = wind_ava[wind_ava['energy'] > wind_ava['energy'].quantile(2/3)]

X_low = low_energy.drop(columns='energy')
y_low = low_energy['energy']

X_high = high_energy.drop(columns='energy')
y_high = high_energy['energy']

# medir el rendimiento en el conjunto de datos con energía baja
y_low_pred = best_model.predict(X_low)
rmse_low = np.sqrt(metrics.mean_squared_error(y_low, y_low_pred))
r2_low = best_model.score(X_low, y_low)

# medir el rendimiento en el conjunto de datos con energía alta
y_high_pred = best_model.predict(X_high)
rmse_high = np.sqrt(metrics.mean_squared_error(y_high, y_high_pred))
r2_high = best_model.score(X_high, y_high)

print(f"RMSE en energía baja: {rmse_low}")
print(f"R2 en energía baja: {r2_low}")

print(f"RMSE en energía alta: {rmse_high}")
print(f"R2 en energía alta: {r2_high}")


### Convertir el problema en uno de clasificacion


In [ ]:
# ETIQUETAR LOS DATOS COMO LOW Y HIGH energy
# añadir una columna al conjunto de datos original que indique si la energía es baja o alta
wind_ava['energy_class'] = 'high'
wind_ava.loc[wind_ava['energy'] < wind_ava['energy'].quantile(1/3), 'energy_class'] = 'low'
# eliminar la columna energy
wind_ava.drop(columns='energy', inplace=True)

wind_ava.head()


#### KNN